# Learning from AirBnb Data

Starting with Tom Lee's CSV from : https://s3.amazonaws.com/tomslee-airbnb-data-2/washington.zip, Let's see if we can answer a few questions :

1. Are the listing ID's unique - that is, the dataset has one entry (row) per room ID?
2. In this dataset, what is the maximum number of listings that any host has?
3. Can we predict the price given some features of the listing (an exercise in K-Nearest-Neighbors supervised machine learning and hyperparameters (N) tuning (optimization) and feature selection - find out which ones really help you build a good model)?

The learning that enabled this came from DataQuest's online course..

In [2]:
# Step one, read in the CSV and get a sense of the dataframe - do you need to do any cleaning etc (KNN is not forgiving)
import pandas as pd
dc_listings = pd.read_csv("tomslee_airbnb_washington_1433_2017-07-11.csv")  # did you know you could type
                                  # a few letters of the name and hit TAB and Jupyter will fill it in?
dc_listings.head(2)

,room_id,survey_id,host_id,room_type,country,city,borough,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,bathrooms,price,minstay,last_modified,latitude,longitude,location
0,3732219,1433,280636,Shared room,NaN,Washington,NaN,Columbia Heights,0,0.0,3,1.0,NaN,129.0,NaN,2017-07-11 08:53:56.381540,38.931081,-77.030618,0101000020E6100000D02A33A5F54153C0A77686A92D77...
1,15087225,1433,90860645,Shared room,NaN,Washington,NaN,Brentwood,6,5.0,4,1.0,NaN,118.0,NaN,2017-07-11 08:53:55.616987,38.908054,-77.003306,0101000020E61000005B785E2A364053C041800C1D3B74...


As a novice data-science practitioner, some questions arise rightaway :

- What if I want to use the room_id as the index for each row? Is that possible? (Yes)
- What if the price, which, in these two rows, looks very well behaved actually had commas and $ characters? Is it possible to automatically convert such "almost truly numeric" columns into truly numeric ones automatically? (A fact of data-science life that, to their credit, DataQuest does drill into you, is that work involves a lot of drudgery - data cleaning)
- Is it possible to (again, automatically) reduce the dataframe to just the numeric columns that really lend themselves to some kind of predictive analysis?

For a start, let's make sure that room_id does have as many unique values as the length of the df :

In [4]:
dc_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8237 entries, 0 to 8236
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   room_id               8237 non-null   int64  
 1   survey_id             8237 non-null   int64  
 2   host_id               8237 non-null   int64  
 3   room_type             8237 non-null   object 
 4   country               0 non-null      float64
 5   city                  8237 non-null   object 
 6   borough               0 non-null      float64
 7   neighborhood          8237 non-null   object 
 8   reviews               8237 non-null   int64  
 9   overall_satisfaction  8237 non-null   float64
 10  accommodates          8237 non-null   int64  
 11  bedrooms              8237 non-null   float64
 12  bathrooms             0 non-null      float64
 13  price                 8237 non-null   float64
 14  minstay               0 non-null      float64
 15  last_modified        

Clearly, this dataset has been given some attention already - because the columns seem to be all-or-nothing. The real life - especially where one is using data that is available-but-not-advertised, is likely to require much more work.

First off, can we set_index to room_id? How many unique room_id's are there?

In [5]:
len( dc_listings['room_id'].unique() )

8235

Clearly, that's a bit depressing :) We were hoping for 8237. What happened?